In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers

c:\Users\sriva\OneDrive\Desktop\End-to-End-Medical-Chatbot-using-Llama-2\mchatvenv\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY = "eeed37e5-7364-4d77-861d-b40a87ed4bc8"
PINECONE_API_ENV = "gcp-starter"

In [3]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,glob="*.pdf",loader_cls= PyPDFLoader)
    documents = loader.load()
    return documents 

In [4]:
doc = load_pdf("../data/")

In [6]:
#doc

In [7]:
#created text chunks
def text_split(doc):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(doc)
    return text_chunks
    

In [8]:
text_chunks = text_split(doc)

In [10]:
len(text_chunks)

7020

In [11]:
#download embedding model
def download_huggingface_embeddings():
    embedding_model = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embedding_model

In [12]:
embed_model = download_huggingface_embeddings()

.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<?, ?B/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<?, ?B/s] 
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 2.65MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<?, ?B/s] 
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 192kB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:15<00:00, 5.86MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<?, ?B/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 27.9kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 978kB/s]
tokenizer_config.json: 100%|██████████| 350/350 [00:00<?, ?B/s] 
train_script.py: 100%|██████████| 13.2k/13.2k [00:00<?, ?B/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 7.14MB/s]
modules.json: 100%|██████████| 349/349 [00:00<?, ?B/s] 


In [14]:
query_embd= embed_model.embed_query("Hello World")
print(len(query_embd))

384


In [16]:
pinecone.init(api_key  = PINECONE_API_KEY, environment= PINECONE_API_ENV)
index_name = "medical-chatbot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embed_model, index_name=index_name)

In [18]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embed_model)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE", metadata={}), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drug

In [19]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [20]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [22]:
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [23]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [24]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

Response :  Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
Response :  Asthma is a chronic respiratory disease characterized by inflammation of the airways, which can cause recurring episodes of wheezing, coughing, chest tightness, and shortness of breath.
